## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [1]:
!pip install cassandra-driver

In [2]:
import pandas as pd
import numpy as np
import cassandra
import csv
from cassandra.cluster import Cluster


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [3]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries

try:
    # we have created created an object ans connected to local cassandra instance
    cluster_connect = Cluster(['127.0.0.1'],port=6000)
    
    #Now create a session to establish the connection
    Cassandra_con = cluster_connect.connect()

except Exception as exp:
    print(exp)
    

#### Create & Set Keyspace

In [4]:
try: 
    Cassandra_con.execute("""select * from event_data""")
except Exception as exp:
    print(exp)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


In [5]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
try:
    #Creating keyspace
    Cassandra_con.execute(" CREATE KEYSPACE IF NOT EXISTS musicdb WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")
    #Setting keyspace
    Cassandra_con.set_keyspace('musicdb')
except Exception as e:
    print(e)

In [6]:
Text = pd.read_csv("event_data.csv")
#to load first 5 rows of data
Text.head()

,artist_name,fname,gender,item_in_session_number,lname,length,level,location,session_number,song_title,user_id
0,Pavement,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",345,Mercy:The Laundromat,10
1,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",438,Horn Concerto No. 4 in E flat K495: II. Romanc...,53
2,Gary Allan,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",438,Nothing On But The Radio,53
3,Charttraxx Karaoke,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",438,Fireflies,53
4,The Libertines,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",389,The Good Old Days,29


In [7]:
Text.shape

(6820, 11)

## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [8]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command   
queryNumber1 = """CREATE TABLE IF NOT EXISTS music_lib1 
                (artist_name text, 
                song_title text, 
                length decimal, 
                session_number int,
                item_in_session_number int,
                PRIMARY KEY (session_number, item_in_session_number))"""
try:
    Cassandra_con.execute(queryNumber1)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    count1 = 0
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        queryNumber2 = """INSERT INTO music_lib1(
                artist_name, 
                song_title, 
                length, 
                session_number,
                item_in_session_number)
                VALUES
                (%s, %s, %s, %s, %s)"""
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        artist_name, song_title, length, session_number, item_in_session_number = row[0], row[9], float(row[5]), int(row[8]), int(row[3])
        Cassandra_con.execute(queryNumber2, (artist_name, song_title, length, session_number, item_in_session_number))
        count1 += 1
print("No. of rows =", count1)

No. of rows = 6820


### Validate our Data Model using a SELECT

In [11]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
queryNumber3 = 'SELECT artist_name, song_title, length from music_lib1 where session_number=338 and item_in_session_number = 4';

try:
    rows1 = Cassandra_con.execute(queryNumber3)
except Exception as exp:
    print(exp)
    
for j in rows1:
    print (j.artist_name, j.song_title, j.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [15]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command                   
queryNumber4 = """CREATE TABLE IF NOT EXISTS music_lib2 (
                artist_name text, 
                song_title text, 
                session_number int,
                fname text, 
                lname text,
                item_in_session_number int,
                user_id int,
                PRIMARY KEY ((user_id, session_number), item_in_session_number))"""
try:
    Cassandra_con.execute(queryNumber4)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [16]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    count2 = 0
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        queryNumber5 = """INSERT INTO music_lib2 (
                artist_name, 
                song_title, 
                session_number,
                fname, 
                lname,
                item_in_session_number,
                user_id)
                VALUES
                (%s, %s, %s, %s, %s, %s, %s)"""
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        artist_name, song_title, session_number, fname, lname, item_in_session_number, user_id = str(row[0]),str(row[9]),int(row[8]),str(row[1]),str(row[4]),int(row[3]),int(row[10])
        Cassandra_con.execute(queryNumber5, (artist_name, song_title, session_number, fname, lname, item_in_session_number, user_id))
        count2+=1

print("No. of rows =", count2)

No. of rows = 6820


### Validate our Data Model using a SELECT

In [17]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
queryNumber6 = """SELECT artist_name, song_title, fname, lname 
                FROM music_lib2 
                WHERE user_id = 10 AND session_number = 182"""

try:
    rows2 = Cassandra_con.execute(queryNumber6)
except Exception as exp:
    print(exp)
    
for h in rows2:
    print (h.artist_name, h.song_title, h.fname, h.lname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [18]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command           
queryNumber7 = """CREATE TABLE IF NOT EXISTS music_lib3 (
                fname text, 
                lname text, 
                song_title text, 
                user_id int,
                PRIMARY KEY (user_id, song_title))"""
try:
    Cassandra_con.execute(queryNumber7)
except Exception as exp:
    print(exp)

### Let's insert our data into of table

In [19]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    count3 = 0
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        queryNumber8 = """INSERT INTO music_lib3( 
                fname, 
                lname,
                song_title,
                user_id)
                VALUES
                (%s, %s, %s, %s)"""
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        song_title, fname, lname, user_id = str(row[9]),str(row[1]),str(row[4]),int(row[10])
        Cassandra_con.execute(queryNumber8, (fname, lname, song_title, user_id))
        count3+=1
    
print("No. of rows in table 3 = ", count3)

No. of rows in table 3 =  6820


### Validate our Data Model using a SELECT

In [20]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
queryNumber9 = "SELECT fname + ' ' + lname AS user_name FROM music_lib3 WHERE song_title = 'All Hands Against His Own' ALLOW FILTERING;"

try:
    rows3 = Cassandra_con.execute(queryNumber9)
except Exception as exp:
    print(exp)
    
for row in rows3:
    print (row.user_name)

Sara Johnson
Tegan Levine
Jacqueline Lynch


### Drop the tables before closing out the sessions

In [25]:
queryNumber10 = "DROP TABLE music_lib1;"
try:
    rows = Cassandra_con.execute(queryNumber10)
except Exception as exp:
    print(exp)

In [27]:
queryNumber11 = "DROP TABLE music_lib2;"
try:
    rows = Cassandra_con.execute(queryNumber11)
except Exception as exp:
    print(exp)

In [28]:
queryNumber12 = "DROP TABLE music_lib3;"
try:
    rows = Cassandra_con.execute(queryNumber12)
except Exception as exp:
    print(exp)

### Close the session and cluster connection¶

In [29]:
cluster_connect.shutdown()
Cassandra_con.shutdown()